In [ ]:
!pip install fasttext

In [ ]:
!pip install numpy==1.23.5

In [ ]:
import numpy as np
assert str(np.__version__)== '1.23.5'
import fasttext
import pandas as pd

In [ ]:
data_frame = pd.read_csv('/content/main_dataset.csv')
data_frame

,toxic,comment
0,1.0,убивать сразу сучек
1,1.0,хорошая попа вдуть
2,1.0,"из вашей инецеативы долбоебы, государство може..."
3,1.0,"ну а что,пизда-сильная организация ...."
4,1.0,"ох, пиздец,прошу за слэнг сейчас придётся адми..."
...,...,...
34407,1.0,Вонючий совковый скот прибежал и ноет. А вот и...
34408,1.0,А кого любить? Гоблина тупорылого что-ли? Или ...
34409,0.0,"Посмотрел Утомленных солнцем 2. И оказалось, ч..."
34410,1.0,КРЫМОТРЕД НАРУШАЕТ ПРАВИЛА РАЗДЕЛА Т.К В НЕМ Н...


In [ ]:
import re

def label_transform(label_numb):
  if label_numb == 1.0:
    return 'negative'
  else:
     return 'positive'

def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^\а-яА-ЯЁё]', ' ', text)
    text = text.strip()
    return text

In [ ]:
data_frame["prepare_comment"] = data_frame.apply(lambda row: "__label__"+label_transform(row.toxic) + ' '+preprocess(row.comment), axis = 1)

In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(data_frame, random_state=42, test_size=0.2)
test_df, val_df = train_test_split(test_df, random_state=42, test_size=0.5)

In [ ]:
data = train_df['prepare_comment'].to_list()
with open("train.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(data))

In [ ]:
model = fasttext.train_supervised("train.txt", lr = 0.1,  epoch = 10, wordNgrams=3, dim=200, loss='softmax' )

In [ ]:
model.save_model('model_fasttext.bin')

In [ ]:
labels_test, probs_test = model.predict(test_df['prepare_comment'].to_list())
classes_test_df = pd.DataFrame({'classes' : labels_test})
classes_test_df['classes'] = classes_test_df['classes'].apply(lambda x: int(x[0]!="__label__positive"))

labels_train, probs_train = model.predict(train_df['prepare_comment'].to_list())
classes_train_df = pd.DataFrame({'classes' : labels_train})
classes_train_df['classes'] = classes_train_df['classes'].apply(lambda x: int(x[0]!="__label__positive"))

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
print("accuracy_train:  ", accuracy_score(train_df['toxic'], classes_train_df['classes']))
print("precision__train:  ", precision_score(train_df['toxic'], classes_train_df['classes']))
print("recall_train:  ", recall_score(train_df['toxic'], classes_train_df['classes']))

print('\n')
print("accuracy_test:  ", accuracy_score(test_df['toxic'], classes_test_df['classes']))
print("precision__test:  ", precision_score(test_df['toxic'], classes_test_df['classes']))
print("recall_test:  ", recall_score(test_df['toxic'], classes_test_df['classes']))


accuracy_train:   0.9807839006138981
precision__train:   0.9872699122656116
recall_train:   0.9678725018972932


accuracy_test:   0.8761987794245859
precision__test:   0.9045801526717557
recall_test:   0.797442799461642
